In [9]:
using JuMP
import Gurobi
using CSV, DataFrames
using Random
import Test

In [8]:
data = CSV.read("RoofDummyData.csv",DataFrame)
data
#nrow(data) = 24
#ncol(data) = 5

Row,Column1,XC60_Gent,XC60_China,Kuga,Mondeo
,String15,Int64,Int64,Int64,Int64
1,1/1/2024,5668,5952,2012,924
2,1/2/2024,3916,3744,2128,800
3,1/3/2024,5312,1552,2632,964
4,1/4/2024,6720,3032,1740,768
5,1/5/2024,4092,2372,1540,648
6,1/6/2024,3108,2292,2292,848
7,1/7/2024,4656,1568,2920,828
8,1/8/2024,4772,612,3256,764
9,1/9/2024,3408,3988,2288,720


In [16]:
Random.seed!(1234) # set seed

num_products_p = ncol(data) - 1  #to not include the time columnindex
time_horizon_T = nrow(data)
product_names = names(data)[2:end]
println(product_names)

#Initialise demand
demand_D = [] #Initialise 
for col in 2:ncol(data) #Extract each column as a seperate array and store it
    push!(demand_D, data[:,col])
end
println(demand_D)

#Initialise workdays 
workdays_n = [] #Initialise 
for row in 1:time_horizon_T
    push!(workdays_n, rand(20:23)) #push a random workday value between 20 to 23
end
println(workdays_n)

#Initialise productivity
productivity_K = [] 
for col in 2:ncol(data)
    push!(productivity_K, rand(10:12))
end    
println(productivity_K)

# Initialise Costs
cost_hiring_cH = 5882.60    # Hiring cost per worker
cost_firing_cF = 857.23    # Firing cost per worker
cost_inventory_cI = 9.05 # Inventory holding cost per unit
cost_labour_cR = 233.30  # Cost of Labour per production unit
cost_overtime_cO = 349.89 # Cost of Overtime per overtime unit
cost_backlogging_cB = 135.31  #Cost of Backlogging per overtime unit 
#code ends here

["XC60_Gent", "XC60_China", "Kuga", "Mondeo"]
Any[[5668, 3916, 5312, 6720, 4092, 3108, 4656, 4772, 3408, 2936, 8284, 4516], [5952, 3744, 1552, 3032, 2372, 2292, 1568, 612, 3988, 2540, 6680, 3260], [2012, 2128, 2632, 1740, 1540, 2292, 2920, 3256, 2288, 2424, 2228, 2660], [924, 800, 964, 768, 648, 848, 828, 764, 720, 448, 700, 568]]
Any[21, 22, 20, 23, 21, 21, 23, 23, 21, 22, 22, 22]
Any[10, 10, 11, 12]


135.31

In [11]:
# Initialize the model
model = Model(Gurobi.Optimizer)

# Variables
@variable(model, workerlevel_W[1:num_products_p,1:time_horizon_T] >= 0)       # Workers
@variable(model, hired_H[1:num_products_p,1:time_horizon_T] >= 0)       # Hired workers
@variable(model, fired_F[1:num_products_p,1:time_horizon_T] >= 0)       # Fired workers
@variable(model, inventory_I[1:num_products_p,1:time_horizon_T] >= 0)       # Inventory as integer variables
@variable(model, production_P[1:num_products_p,1:time_horizon_T] >= 0)       # Production
@variable(model, overtime_O[1:num_products_p,1:time_horizon_T] >= 0)       # Overtime
@variable(model, backlogging_B[1:num_products_p,1:time_horizon_T] >= 0)        # Production

# Objective function: Minimize total cost
@objective(model, Min, sum(cost_hiring_cH*hired_H + cost_firing_cF*fired_F 
+ cost_inventory_cI*inventory_I + cost_labour_cR*production_P 
+ cost_overtime_cO*overtime_O + cost_backlogging_cB*backlogging_B))

# Constraints

#only for period 1, W annd I-B eqns
for p in 1:num_products_p
    @constraint(model, workerlevel_W[p,1] == hired_H[p,1] - fired_F[p,1])
    @constraint(model, inventory_I[p,1] - backlogging_B[p,1] == production_P[p,1] - demand_D[p][1])
end    

#P equations
for p in 1:num_products_p
    for t in 2:time_horizon_T
        @constraint(model, workerlevel_W[p,t] 
        == workerlevel_W[p,t-1] + hired_H[p,t] - fired_F[p,t])
        @constraint(model, inventory_I[p,t] - backlogging_B[p,t] 
        == inventory_I[p,t-1] - backlogging_B[p,t-1] 
        + production_P[p,t] - demand_D[p][t])
    end
end

#for rest of period, W and I-B equations
for p in 1:num_products_p
    for t in 1:time_horizon_T
        @constraint(model, production_P[p,t] 
        == productivity_K[p] * workdays_n[t] * workerlevel_W[p,t])
    end    
end

# Solve the model
optimize!(model)

#retrieve Values
println("Objective value: ", objective_value(model))
println("Workers: ", round.(Int,value.(workerlevel_W)))
println("Hired: ", round.(Int,value.(hired_H)))
println("Fired: ", round.(Int,value.(fired_F)))
println("Inventory: ", round.(Int,value.(inventory_I)))
println("Production: ", round.(Int,value.(production_P)))
println("Overtime: ", round.(Int,value.(overtime_O)))
println("Backlogging: ", round.(Int,value.(backlogging_B)))

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2528347
Academic license 2528347 - for non-commercial use only - registered to jo___@mymail.sutd.edu.sg
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2528347 - for non-commercial use only - registered to jo___@mymail.sutd.edu.sg
Optimize a model with 144 rows, 336 columns and 516 nonzeros
Model fingerprint: 0x7f98dbf1
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [9e+00, 6e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 8e+03]
Presolve removed 56 rows and 116 columns
Presolve time: 0.01s
Presolved: 88 rows, 220 columns, 388 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.22175